In [1]:
import sys
sys.path.insert(1,'D:\Github\DeepLake-Langchain')
import credentials
import os
os.environ["OPENAI_API_KEY"] = credentials.openai

In [10]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "what is the word to replace the following: {word}"

llm = OpenAI(model = "text-davinci-003",temperature=0.0)

llm_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain("artificial")

{'word': 'artificial', 'text': '\n\nsynthetic'}

In [11]:
input_list = [
    {"word":"artificial"},
    {"word":"intelligence"},
    {"word":"robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nsynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nautomaton'}]

In [12]:
llm_chain.generate(input_list=input_list)


LLMResult(generations=[[Generation(text='\n\nsynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nautomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 30, 'total_tokens': 44}, 'model_name': 'text-davinci-003'}, run=RunInfo(run_id=UUID('d1506519-f450-41dd-9fee-b94f63f4667f')))

In [13]:
prompt_template = "Looking at the context of {context}. What is the appropriate word to replace the following: {word}?"


llm_chain  =LLMChain(prompt=PromptTemplate(template = prompt_template,input_variables=["word","context"]), llm=llm)
llm_chain.predict(word="fan",context = "object")

'\n\nventilator'

In [14]:
llm_chain.predict(word="fan",context = "humans")

'\n\nAdmirer'

### Chain with Parsers

In [15]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = "List all possible words as substitute for 'artificial' as comma seperated."

In [16]:
llm_chain = LLMChain(llm = llm, 
                     prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]))
llm_chain.predict_and_parse()

['Synthetic',
 'manufactured',
 'man-made',
 'simulated',
 'fabricated',
 'mechanical',
 'contrived',
 'manufactured',
 'computerized',
 'robotic',
 'programmed',
 'automated',
 ' engineered',
 ' artificial intelligence',
 ' manufactured',
 ' lab-created',
 ' manufactured',
 ' constructed',
 ' imitation',
 ' unreal',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured',
 ' manufactured

### Conversational Chain

In [19]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(llm=llm, prompt =PromptTemplate(template=template, input_variables=["history","input"],output_parser=output_parser),memory=ConversationBufferMemory())

conversation.predict_and_parse(input = "Answer briefly. write the first 3 options")

['Synthetic', 'manufactured', 'simulated']

In [20]:
conversation.predict_and_parse(input="And the next 4?")

['Imitated', 'contrived', 'fabricated', 'man-made']

### Simple Sequential Chain


In [21]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [28]:
llm = OpenAI(temperature=0.7)
template = """
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play
"""
prompt_template = PromptTemplate(template=template, input_variables=['title'])
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template,verbose=True)

In [29]:
llm = OpenAI(temperature=0.7)
template = """
You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
"""
prompt_template = PromptTemplate(template=template, input_variables=['synopsis'])
review_chain = LLMChain(llm=llm, prompt=prompt_template,verbose=True)

In [30]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [31]:
review = overall_chain.run("Targedy at sunset on the beach")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: Targedy at sunset on the beach
Playwright: This is a synopsis for the above play


> Finished chain.

Targedy at Sunset on the Beach is a whimsical comedy about the lives of four twenty-somethings living on the beach in a small, coastal town. They come from different backgrounds and have different dreams, but they all share one thing: a desire to make the most of every minute of their lives.

The play follows the four friends as they experience the joys and heartaches of life, from love and relationships, to career aspirations, to facing fears and growing pains. As the sun sets on each day, the friends share their stories, laugh, cry, and ultimately find comfort and strength in one another.

The action culminates in a climactic moment at sunset on the beach where the friends each come to terms with thei

In [27]:
review

'\nThe play “At the Beach” is a beautiful reminder of the power of love and family in times of hardship. The story follows a family on vacation at the beach, where they are enjoying the perfect time for relaxation and fun. However, tragedy strikes in the midst of the sunset when one of the family members is involved in a fatal accident. The play follows the family’s journey to healing and understanding the tragedy that has befallen them. It is a captivating story about how a family is able to come together and persevere through the darkness of grief and loss. The play is full of emotion and heart-wrenching moments that will stay with viewers long after the curtains close. The performances of the actors are powerful and moving, and they bring the story to life in a way that is both touching and moving. “At the Beach” is an emotional journey that will leave a lasting impression on its viewers.'